# Construct the Dataset

## Table of Contents
- [Split scans to train, validate, and test](#split)
- [Filter scans by nodule diameter](#filter)
- [Generate dataset](#generatedata)
    - [Positive examples](#pos)
    - [Negative examples](#neg)

Now that we have developed our toolbox, DLToolboxImg, we are ready to build our dataset. The dataset will be derived from the LIDC dataset, with some modifications.

From the ct-scan volumes provided by LIDC, we will extract 3d-patches of ct=scans. Those patches will serve as the input. The label will be a bolean mask where each pixel is classified as part of a nodule region or not.


4Roadmap:
1. split scans to train, validate, test
2. Generate patches that contain a nodule:
    - given a scan: find nodules centroid, find ctscan volume, find mask. 
    - extract a 52x52x52 patch around the nodule from the scan and associated mask
    - save image in folder
    - save path in a .csv file
    

## Split scans to train, validate, and test
<a id="split"></a>

## Filter scans by nodule diameter
<a id="filter"></a>

## Generate dataset 
<a id="generatedata"></a>

### Positive examples
<a id="pos"></a>

### Negative examples
<a id="neg"></a>

In [ ]:
this is where 